In [1]:
import diff_gaussian_rasterization
import torch
import numpy as np
import pyquaternion
import os
import math
# cuda_launch_blocking = True
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"


In [2]:
device = torch.device("cpu")

# generate a random point cloud
N = 1000
means3D = torch.rand(N, 3, dtype=torch.float32, device=device) * 4 - 2
means2D = torch.zeros(N, 2, dtype=torch.float32, device=device)
opacities = torch.ones(N, 1, dtype=torch.float32, device=device)
scales = torch.ones(N, 3, dtype=torch.float32, device=device) * 0.1
rotations = torch.from_numpy(np.stack([pyquaternion.Quaternion().elements for _ in range(N)])).to(torch.float32).to(device)
colors_precomp = torch.rand(N, 3, dtype=torch.float32, device=device)


In [3]:
# generate the projection matrix (orthographic)

def gen_orthographic_matrix(left: float, right: float, bottom: float, top: float, near: float, far: float):
    """Generate orthographic projection matrix.

    Args:
        left (float): left plane
        right (float): right plane
        bottom (float): bottom plane
        top (float): top plane
        near (float): near plane
        far (float): far plane

    Returns:
        torch.Tensor: orthographic projection matrix
    """
    return torch.tensor([
        [2 / (right - left), 0, 0, -(right + left) / (right - left)],
        [0, 2 / (top - bottom), 0, -(top + bottom) / (top - bottom)],
        [0, 0, -2 / (far - near), -(far + near) / (far - near)],
        [0, 0, 0, 1],
    ])

In [4]:
FoVx = 90
FoVy = 90

raster_settings = diff_gaussian_rasterization.GaussianRasterizationSettings(
    image_height=512,
    image_width=512,
    tanfovx=math.tan(FoVx * 0.5),
    tanfovy=math.tan(FoVy * 0.5),
    bg=torch.zeros(4, dtype=torch.float32, device=device),
    scale_modifier=1.0,
    viewmatrix=torch.eye(4, dtype=torch.float32, device=device),
    projmatrix=gen_orthographic_matrix(-1, 1, -1, 1, 0.1, 10),
    sh_degree=1,
    campos=torch.tensor([0, 0, 0], dtype=torch.float32, device=device),
    prefiltered=False,
    debug=True,
)

In [5]:
rasterizer = diff_gaussian_rasterization.GaussianRasterizer(raster_settings=raster_settings)


In [6]:
print("means3D", means3D.shape)
print("means2D", means2D.shape)
print("scales", scales.shape)
print("rotations", rotations.shape)
print("opacities", opacities.shape)
print("colors_precomp", colors_precomp.shape)

means3D torch.Size([1000, 3])
means2D torch.Size([1000, 2])
scales torch.Size([1000, 3])
rotations torch.Size([1000, 4])
opacities torch.Size([1000, 1])
colors_precomp torch.Size([1000, 3])


In [7]:
rendered_image, radii = rasterizer(
    means3D=means3D,
    means2D=means2D,
    colors_precomp=colors_precomp,
    opacities=opacities,
    scales=scales,
    rotations=rotations,
)

radii size: 1000
P: 1000, D: 1, M: 0
radii is null: 0
Preprocessing points
means3D[0]: 1.857995 
radii[0]: 0 
means3D[1]: -0.656337 
radii[1]: 0 
means3D[2]: 1.237953 
radii[2]: 0 
means3D[3]: -0.448509 
radii[3]: 0 
means3D[4]: 1.011580 
radii[4]: 0 
means3D[5]: -1.860926 
radii[5]: 0 
means3D[6]: -0.468488 
radii[6]: 0 
means3D[7]: -1.588723 
radii[7]: 0 
means3D[8]: -0.721837 
radii[8]: 0 
means3D[9]: -1.664189 
radii[9]: 0 
means3D[10]: -1.684946 
radii[10]: 0 
means3D[11]: -0.587982 
radii[11]: 0 
means3D[12]: -0.489346 
radii[12]: 0 
means3D[13]: -1.672072 
radii[13]: 0 
means3D[14]: -0.764885 
radii[14]: 0 
means3D[15]: 1.913102 
radii[15]: 0 
means3D[16]: -0.117934 
radii[16]: 0 
means3D[17]: -1.058203 
radii[17]: 0 
means3D[18]: 0.880491 
radii[18]: 0 
means3D[19]: 0.289537 
radii[19]: 0 
means3D[20]: 1.852416 
radii[20]: 0 
means3D[21]: -0.433097 
radii[21]: 0 
means3D[22]: 0.208628 
radii[22]: 0 
means3D[23]: -0.081705 
radii[23]: 0 
means3D[24]: -1.451101 
radii[24]: 0 
mea


[CUDA ERROR] in /home/fidgetsinner/projects/diff-gaussian-rasterization/cuda_rasterizer/rasterizer_impl.cu
Line 252: an illegal memory access was encountered

RuntimeError: an illegal memory access was encountered